In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ESG_Project/Dataset/labeled_news_with_sentiments.csv')
df.dtypes

text                   object
predicted_sentiment     int64
dtype: object

In [ ]:
df.tail(2)

,text,predicted_sentiment
3454,Nestlé is Planting 10 Million Trees in Austral...,1
3455,Costco to Set New Emission Cut Targets in Deal...,0


In [ ]:
df['predicted_sentiment'].unique()

array([1, 2, 0])

In [ ]:
x=list(df['text'])
y=list(df['predicted_sentiment'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)

In [ ]:
x_train

['UK and Singapore Enhance Cooperation in Sustainable Finance and FinTech',
 'GHGSat Launches World’s First Satellite Into Orbit That Can Detect and Share CO2 Data',
 'TDK Ventures Launches $150 Million Fund to Invest in Energy and Climate Tech Startups',
 'Deloitte Leads the Way in ESG & Sustainability Services, Verdantix Report Finds',
 'IBM Envizi Enhances ESG Planning and Forecasting with AI',
 'BlackRock Expands NZ Climate Infrastructure Team, Appoints Will Thomson as Managing Director',
 'Mexico Elects Claudia Sheinbaum as First Woman President',
 'PUMA Concludes Season One of Industry-first Sustainability Report Podcast Series',
 'TotalEnergies and Paprec Develop the First French Value Chain for Advanced Recycling of Plastic Film',
 'DTE To Retire Coal Plants by 2032, Invest $11 Billion in Clean Energy Push',
 'The English Football Premier League Takes Big Step in Sustainability with New Commitment',
 'Fils and Telr Partner to Drive Sustainability in Finance and Payments',
 'Fil

In [ ]:
!pip install transformers

In [ ]:

# from transformers import DistilBertTokenizerFast
# tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_encodings=tokenizer(x_train,truncation=True,padding=True)
test_encodings= tokenizer(x_test,truncation=True,padding=True)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
import tensorflow as tf

train_dataset=tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))



In [ ]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(32,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(32,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:
# !pip install transformers[torch]

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

In [ ]:
!pip install transformers==4.18.0

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

df = pd.read_csv('/content/drive/MyDrive/ESG_Project/Dataset/labeled_news_with_sentiments.csv')


print(df['predicted_sentiment'].unique())


train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize text data
def tokenize_data(df):
    return tokenizer(
        df['text'].tolist(),
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='tf'
    )

train_encodings = tokenize_data(train_df)
test_encodings = tokenize_data(test_df)

train_labels = train_df['predicted_sentiment'].tolist()
test_labels = test_df['predicted_sentiment'].tolist()

# Create TensorFlow datasets
def create_dataset(encodings, labels):
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    )).shuffle(len(labels)).batch(8)

train_dataset = create_dataset(train_encodings, train_labels)
test_dataset = create_dataset(test_encodings, test_labels)

# Load pre-trained DistilBERT model for sequence classification
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(df['predicted_sentiment'].unique()))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

# Save the model
model.save_pretrained('./saved_model')




[1 2 0]


Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_57', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/3
346/346 [==============================] - 1182s 3s/step - loss: 0.3956 - accuracy: 0.8263 - val_loss: 0.2839 - val_accuracy: 0.8974
Epoch 2/3
346/346 [==============================] - 1181s 3s/step - loss: 0.1291 - accuracy: 0.9548 - val_loss: 0.2714 - val_accuracy: 0.9408
Epoch 3/3
87/87 [==============================] - 49s 567ms/step - loss: 0.2077 - accuracy: 0.9393
Loss: 0.20772114396095276
Accuracy: 0.939306378364563


In [ ]:
model.save_pretrained('/content/drive/MyDrive/ESG_Project/saved_model')

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/ESG_Project/DistilBert_Tokenizer')

('/content/drive/MyDrive/ESG_Project/DistilBert_Tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/ESG_Project/DistilBert_Tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/ESG_Project/DistilBert_Tokenizer/vocab.txt',
 '/content/drive/MyDrive/ESG_Project/DistilBert_Tokenizer/added_tokens.json')